In [1]:
import pandas as pd
import numpy as np
import pymssql

In [2]:
mssql_conn = pymssql.connect(server='epdm', user='sa', password='Pdmepdme1!',database='ePDM_Elcam')

In [3]:
query='select * from dbo.UserRevs'
df=pd.read_sql(query,mssql_conn)

c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
#organize the data by DocuementID
df=df.sort_values(by=['DocumentID'])
#filter out the data where date  is greater than 2018-01-01
df=df[df['Date']>='2018-01-01']
df

,UserRevID,DocumentID,RevNr,Date,UserID,Comment,UserComment
105813,118360,532,10,2020-12-31 11:41:10.810,55,B,
91660,104014,535,10,2020-03-10 09:34:46.720,55,C,
102334,114850,535,16,2020-10-07 08:34:25.550,55,D,
91664,104018,537,28,2020-03-10 09:34:51.143,55,E,
91666,104020,539,8,2020-03-10 09:34:52.950,55,A,
...,...,...,...,...,...,...,...
133380,146199,540430,3,2023-02-02 08:35:33.047,705,None,
133410,146229,540439,7,2023-02-02 15:46:00.770,23,1,
133393,146212,540455,3,2023-02-02 10:27:47.590,82,1,
133397,146216,540491,3,2023-02-02 12:54:52.810,705,None,


In [6]:
query2='select * from dbo.Documents'
df2=pd.read_sql(query2,mssql_conn)


c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
df2

,DocumentID,Filename,LockProject,UserID,LockDomain,LockPath,Busy,Flushed,DefValStored,RevGenCounter,...,ExtensionID,LockDate,UserDocRefsModified,Deleted,Shared,LockViewID,Link,DocTypeID,ObjectTypeID,Flags
0,1,No Document,2,1,None,None,False,0,0,0.0,...,1.0,NaT,False,False,0,None,False,1.0,0,0
1,5,pl020120000.slddrw,2,1,,,False,2,1,NaN,...,3.0,NaT,False,False,1,None,False,1.0,1,0
2,6,pl020100000.slddrw,2,1,,,False,2,1,NaN,...,3.0,NaT,False,False,1,None,False,1.0,1,0
3,7,pl020110003.slddrw,2,1,,,False,2,1,NaN,...,3.0,NaT,False,False,1,None,False,1.0,1,0
4,8,sp000768.sldprt,2,1,,,False,2,1,NaN,...,5.0,NaT,False,False,1,None,False,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468963,540505,D00597617_draft per revE.docx,84626,402,MNENG-NOY-HP,C:\ePDM_Elcam\Validations\Val-1685-Medtronic F...,False,1,1,NaN,...,30.0,2023-02-02 14:40:47.470,False,False,1,2cac5be5-5ef8-4451-b5d3-c198e3ffdc8e,False,NaN,1,0
468964,540506,ECO-23-0023-הורדת התקן לבדיקת עובי למינט עבור ...,2,1,,,False,2,1,NaN,...,30.0,NaT,False,False,1,None,False,1.0,1,0
468965,540507,נספח ECO-23-0023-הורדת התקן לבדיקת עובי למינט ...,2,1,,,False,2,1,NaN,...,28.0,NaT,False,False,1,None,False,1.0,1,0
468966,540508,514024-green.sldprt,93709,258,ENG-HILLELY-HP,C:\ePDM_Elcam\Documents\Sales\Contract Review\...,False,1,1,NaN,...,5.0,2023-02-02 16:16:35.000,False,False,1,be04bc16-dc0e-4cc7-a858-77f52f5ae1c4,False,NaN,1,0


In [15]:
df3=pd.merge(df,df2,on='DocumentID')
#select all rows where the document id is 537
df3[df3['DocumentID']==537]

,UserRevID,DocumentID,RevNr,Date,UserID_x,Comment,UserComment,Filename,LockProject,UserID_y,...,ExtensionID,LockDate,UserDocRefsModified,Deleted,Shared,LockViewID,Link,DocTypeID,ObjectTypeID,Flags
3,104018,537,28,2020-03-10 09:34:51.143,55,E,,tr050204001.sldprt,2,1,...,5.0,NaT,False,False,1,None,False,1.0,1,0
